# Notificación de Error en Pipeline

Propósito: Reunir mensajes de error de tareas fallidas y enviar una notificación por correo con el detalle para triage y resolución.

Rol: alertas operativas en Databricks — facilita la identificación rápida de fallos en las distintas capas del flujo (Bronze/Silver/Gold).

In [0]:
# NOTIFICACIÓN POR CORREO
import json
import smtplib
from email.message import EmailMessage
from io import StringIO

import pandas as pd

In [0]:
# 1. DEFINIR PARÁMETROS (WIDGETS)
dbutils.widgets.text("send_to", "xxxxxx@gmail.com", "Enviar al Correo")
#dbutils.widgets.text("mensaje", "", "Mensaje del Correo")

In [0]:
lista_tasks = ["Capa_Bronze_01", "Capa_Silver_01", "Capa_Golden_01"]

In [0]:
mensaje = ""
errores = []

for task in lista_tasks:
    try:
        err = dbutils.jobs.taskValues.get(taskKey=task, key="error", debugValue=None)
        if err:
            errores.append(f" X {task}: {err}")
    except Exception:
        # La tarea no falló, no se encontró el valor, o no registró mensaje de error.
        pass

if errores:
    mensaje = "Se detectaron errores en las siguientes tareas:\n\n" + "\n".join(errores)
else:
    mensaje = "Se detectó un error en el pipeline, pero no se encontró detalle del error."


In [0]:
# 2. CONFIGURACIÓN Y ENVÍO DEL CORREO

sender_email = "xxxxxx@gmail.com"
sender_password = "xxxx"

receiver_email = dbutils.widgets.get("send_to")  # Obtenido del widget
smtp_server = "smtp.gmail.com"  # Servidor SMTP para Gmail
smtp_port = 587  # Puerto TLS

# Crea el objeto EmailMessage
msg = EmailMessage()
msg.set_content(mensaje)
msg["Subject"] = " X ERROR detectado en Pipeline Bancario"
msg["From"] = sender_email
msg["To"] = receiver_email

try:
    # Conecta al servidor SMTP y envía el correo
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()  # Asegura que la conexión es segura (TLS)
        server.login(sender_email, sender_password)
        server.send_message(msg)
    print(f"Correo de notificación de error enviado a {receiver_email}")
except Exception as e:
    print(f"Error al enviar el correo: {e}")